In [1]:
from typing import List, Dict, Literal, Union
import datetime, time 
import numpy as np 
import pandas as pd
import RPi.GPIO as GPIO

import plotly.express as px

from multiprocessing import Pipe, Process
from multiprocessing.connection import Connection
 
from data_collection.data_collection import LoggerSet, Logger

from components.two_wheels import TwoWheels
from components.webbased_controller import WebbaseTwoWheelsControl
#from components.camera import Camera
from components.two_wheels import TwoWheelsOnCommandV3

from components.ultrasound_dist_sensor import DistanceSensorUrgentStop
%load_ext autoreload
%autoreload 2

/tmp/ipykernel_3448/1997712061.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
logger_set = LoggerSet('../log/temp', True)

In [3]:
web_p, web_out = WebbaseTwoWheelsControl.start()
dist_p, dist_out = DistanceSensorUrgentStop.start(11, 13, 20e-2, logger_set, name='dist_sensor')
wheels_p, wheels_in = TwoWheelsOnCommandV3.start(33, 32, logger_set, name='two_wheels', save_interval=30)

======== Running on http://0.0.0.0:8765 ========
(Press CTRL+C to quit)


/home/kawa/projects/car1/src/components/two_wheels.py:11: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT)


In [4]:
dist_p, dist_out = DistanceSensorUrgentStop.start(11, 13, 20e-2, logger_set, name='dist_sensor')


/home/kawa/projects/car1/src/components/ultrasound_dist_sensor.py:45: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin_trigger, GPIO.OUT)


In [6]:
def main():
    while web_out.poll():
        web_out.recv()

    while dist_out.poll():
        dist_out.recv()

    
    while True: 
        stop = (dist_out.poll() and dist_out.recv())
        
        if stop:
            wheels_in.send({'go':False})
            
            i = 0
            while dist_out.recv():
                web_out.recv() if web_out.poll() else {}
                print(f'{i}: still too close', end = '      \r')
                i += 1 
                
        else: 
        
            command = web_out.recv() if web_out.poll() else {}
            if command:
                wheels_in.send(command)

main()
    

KeyboardInterrupt: 

In [4]:
web_out.recv()

{'.': True}

In [5]:
loggers = logger_set.get_all_logger()

In [6]:
df = loggers[-1].load_as_df(0,1)

In [7]:
df[df['command_update'].apply(lambda x: ((type(x) == dict) and ('go' in x) and x['go']))]

key,command,command_update,going,left,right,time,warnings
idx,,,,,,,
121,"{'go': True, 'V+': False, 'L+': False, 'R+': F...",{'go': True},True,12.5,12.5,2024-02-19 22:22:39.752380,[]
127,"{'go': True, 'V+': False, 'L+': False, 'R+': F...",{'go': True},True,12.5,12.5,2024-02-19 22:22:40.354209,[]
128,"{'go': True, 'V+': False, 'L+': False, 'R+': F...",{'go': True},True,12.5,12.5,2024-02-19 22:22:40.454745,[]
129,"{'go': True, 'V+': False, 'L+': False, 'R+': F...",{'go': True},True,12.5,12.5,2024-02-19 22:22:40.555323,[]
130,"{'go': True, 'V+': False, 'L+': False, 'R+': F...",{'go': True},True,12.5,12.5,2024-02-19 22:22:40.655787,[]
131,"{'go': True, 'V+': False, 'L+': False, 'R+': F...",{'go': True},True,12.5,12.5,2024-02-19 22:22:40.756383,[]
146,"{'go': True, 'V+': False, 'L+': False, 'R+': F...",{'go': True},True,17.5,17.5,2024-02-19 22:22:42.261848,[]
153,"{'go': True, 'V+': False, 'L+': False, 'R+': F...",{'go': True},True,17.5,17.5,2024-02-19 22:22:42.963960,[]
154,"{'go': True, 'V+': False, 'L+': False, 'R+': F...",{'go': True},True,17.5,17.5,2024-02-19 22:22:43.064551,[]


In [15]:
for k, v in  {'go': True}.items():
    print(k,v)

go True


In [17]:
irdf = loggers[0].load_as_df(0,1)

In [19]:
irdf['command'] = irdf['code'].map(ircode2command)

In [20]:
irdf

key,code,sig,time-sent,command
idx,,,,
1,8377431,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2024-02-17 13:46:37.236947,.
2,8377367,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2024-02-17 13:46:38.914300,.
3,8300631,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2024-02-17 13:46:40.511847,.
4,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2024-02-17 13:46:40.727480,.
5,16754775,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2024-02-17 13:46:41.593033,V+
6,16230485,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2024-02-17 13:46:42.330316,.
7,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2024-02-17 13:46:42.543320,.
8,8300631,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2024-02-17 13:46:42.997044,.
9,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2024-02-17 13:46:43.209586,.
